# 합성곱 신경망(CNN)

## 01 합성곱/풀링 계층 구현하기

### 4차원 배열

CNN에서 계층 사이를 흐르는 데이터는 4차원이다.

In [1]:
import numpy as np

# 높이 28, 너비 28, 채널 1개인 데이터 10개를 무작위로 생성
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [2]:
# 10개 중 첫 번째 데이터에 접근
x[0].shape

(1, 28, 28)

In [4]:
# 첫 번째 채널의 공간 데이터에 접근
x[0, 0].shape  # 또는 x[0][0]

(28, 28)

### for문 대신 im2col로 데이터 전개하기

* im2col : 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수, image to column, 이미지에서 행렬로

* 카페(Cafe)나 체이너(Chainer) 등의 딥러닝 프레임워크에서 합성곱 계층 구현할 때 사용

* 필터 적용 영역이 겹치게 되면 im2col이 메모리를 더 많이 소비하는 단점이 있지만, 행렬 계산으로 만들면 선형 대수 라이브러리를 활용해 효율을 높일 수 있다.

### im2col 이용하여 합성곱 계층 구현하기

In [5]:
# im2col(input_data, filter_h, filter_w, stride=1, pad=0)

import sys, os
os.chdir("C:/Users/Soyeon/Desktop/Study/DL-Study/밑시딥/deep-learning-from-scratch/ch07/")
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터의 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [6]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # 축 순서 변경 (N,H,W,C) -> (N,C,H,W)
        
        return out

* 합성곱 계층의 역전파 구현은 col2im 을 사용하면 된다